# Task -1 : Classifying Text into Counter and Non Counter Class using LSTM
### This notebook is used to measure performance of LSTM using random embeddings

In [7]:
import argparse
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, LSTM ,GRU
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input,merge, Convolution1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Bidirectional
from keras import optimizers

import random
import pickle
import numpy as np
import pdb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.model_selection import KFold
from keras.utils import np_utils
import codecs
import operator
import gensim, sklearn
from string import punctuation
from collections import defaultdict
#from batch_gen import batch_gen
import sys
from sklearn.utils import class_weight
import wordninja
from nltk import tokenize as tokenize_nltk
#from my_tokenizer import glove_tokenize
import math
import re
### Preparing the text data
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

# vocab generation
vocab, reverse_vocab = {}, {}
freq = defaultdict(int)
tweets = {}
from sklearn.metrics import confusion_matrix,make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support



#### This function will return the plot of the confusion matrix 
1. input: confusion matrix and target names(class_name)
2. output: plot of confusion matrix 

In [2]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

#### This function is generating the classification report
1. input: ground_truth and predicted outputs
2. output: dataframe containing the results

In [3]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T,avg

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import os
#GLOVE_MODEL_FILE = "../../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
GLOVE_MODEL_FILE="../../../glove.840B.300d.txt"

print(os.path.isfile(GLOVE_MODEL_FILE))

EMBEDDING_DIM = 300


glove_file = GLOVE_MODEL_FILE
tmp_file = get_tmpfile("test_word2vec_300.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)


# In[29]:


word2vec_model = KeyedVectors.load_word2vec_format(tmp_file)

In [8]:
###for fast text word embeddings
word2vec_model = KeyedVectors.load_word2vec_format('../../../wiki.en.vec')


In [35]:
len(word2vec_model.vocab)

2519370

In [10]:
#batches are geenrated for the min batch training environment 
def batch_gen(X, batch_size):
    n_batches = X.shape[0]/float(batch_size)
    n_batches = int(math.ceil(n_batches))
    end = int(X.shape[0]/float(batch_size)) * batch_size
    n = 0
    for i in range(0,n_batches):
        if i < n_batches - 1: 
            batch = X[i*batch_size:(i+1) * batch_size, :]
            yield batch
        
        else:
            batch = X[end: , :]
            n += X[end:, :].shape[0]
            yield batch


In [11]:
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from commen_preprocess import *

### this is tokenization for word embeddings 
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    #words =[ps.stem(word) for word in words]
    return words


....start....cleaning


In [12]:
### embeddings if present for the word ..return the embeddings else return a vector of embeddings vector 0  
def get_embedding(word):
    #return
    try:
        return word2vec_model[word]
    except Exception as e:
        print('Encoding not found: %s' %(word))
        return np.zeros(EMBEDDING_DIM)

#form the embeddings matrix where ith row is the embedding vector of the ith word in the vocab    
def get_embedding_weights():
    embedding = np.zeros((len(vocab) + 1, EMBEDDING_DIM))
    n = 0
    for k, v in vocab.items():
        try:
            embedding[v] = word2vec_model[k]
        except:
            n += 1
            pass
    print("%d embedding missed"%n)
    return embedding

#### comments and corresponding labels collection from the defined pd_train(the dataframe) 
def get_data():
    comments=pd_train['text'].values
    labels=pd_train['class'].values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment

## select those tweets only which have the embeddings present in the word2vec model
#not required in case of random embeddings  
def select_tweets():
    # selects the tweets as in mean_glove_embedding method
    # Processing
    tweet_1 = get_data()
    X, Y = [], []
    tweet_return = []
    for tweet in tweet_1:
        _emb = 0
        words = glove_tokenize(tweet['text'].lower())
        for w in words:
            if w in word2vec_model:  # Check if embeeding there in GLove model
                _emb+=1
        if _emb:   # Not a blank tweet
            tweet_return.append(tweet)
    print('Tweets selected:', len(tweet_return))
    return tweet_return


In [22]:
 
import json
####give the whole complete path of the file along with the filename 
path='../Data/Counterspeech_Dataset.json'
with open(path) as fp:
    train_data = json.load(fp)   

In [21]:
from commen_preprocess import *
## converting class label to counter or non counter depending on the value of the class 0 or 1 
def convert_class_label(input_text):
    if input_text:
        return 'counter'
    else:
        return 'noncounter'

In [23]:
###training datafrane created

pd_train = pd.DataFrame(columns=['id','class','community','category','text'])

###values inserted in the dataframe 
for count, each in enumerate(train_data):
    try:
        pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
    except:
        pass
print('Training Data Loading Completed...')

Training Data Loading Completed...


In [50]:
##generating the voacbs from the text
def gen_vocab(tweets_1):
    # Processing
    vocab_index = 1
    for tweet in tweets_1:
        text = glove_tokenize(tweet['text'].lower())
        #text = ''.join([c for c in text if c not in punctuation])
        #words = text.split()
        #words = [word for word in words if word not in STOPWORDS]
        words=text
        for word in words:
            if word not in vocab:
                vocab[word] = vocab_index
                reverse_vocab[vocab_index] = word       # generate reverse vocab as well
                vocab_index += 1
            freq[word] += 1
    vocab['UNK'] = len(vocab) + 1
    reverse_vocab[len(vocab)] = 'UNK'


###filtering out vocab by taking the top k vocab words 
def filter_vocab(k):
    global freq, vocab
    pdb.set_trace()
    freq_sorted = sorted(list(freq.items()), key=operator.itemgetter(1))
    tokens = freq_sorted[:k]
    vocab = dict(list(zip(tokens, list(range(1, len(tokens) + 1)))))
    vocab['UNK'] = len(vocab) + 1

### generate a X and y for training 
def gen_sequence(tweets_new):
    y_map = {
            'counter': 0,
            'noncounter': 1
             }

    X, y = [], []
    for tweet in tweets_new:
        text = glove_tokenize(tweet['text'].lower())
        #text = ''.join([c for c in text if c not in punctuation])
        #words = text.split()
        #words = [word for word in words if word not in STOPWORDS]
        words=text
        seq, _emb = [], []
        for word in words:
            seq.append(vocab.get(word, vocab['UNK']))
        X.append(seq)
        y.append(y_map[tweet['label']])
    return X, y



###if weights are random then shuffle the weights 

def shuffle_weights(model):
    weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)

    
###MAIN MODEL OF LSTM are defined here    
def lstm_model(sequence_length, embedding_dim,dropout1):
    model_variation = 'GRU'
    print(('Model variation is %s' % model_variation))
    model = Sequential()
    model.add(Embedding(len(vocab)+1, embedding_dim, input_length=sequence_length, trainable=LEARN_EMBEDDINGS))
    #model.add(Dropout(dropout1))#, input_shape=(sequence_length, embedding_dim)))
    model.add(Bidirectional(GRU(HIDDEN)))
    #model.add(GRU(50))
    model.add(Dropout(dropout1))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    #sgd = optimizers.SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    adam=optimizers.Adam(lr=LR, beta_1=0.9, beta_2=0.999, decay=0.01)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    print(model.summary())
    return model
from keras import backend as K

###training the lstm ###  

def train_LSTM(X_temp,sentence_len,model, inp_dim, weights, epochs=10, batch_size=512,INITIALIZE_WEIGHTS_WITH = "random"):
        if INITIALIZE_WEIGHTS_WITH == "glove":
            model.layers[0].set_weights([weights])
        elif INITIALIZE_WEIGHTS_WITH == "random":
            shuffle_weights(model)
        else:
            print("ERROR!")
            return
        for epoch in range(epochs):
            loss_all=0.0
            acc_all=0.0
            count=0.0
            for X_batch in batch_gen(X_temp, batch_size):
                x = X_batch[:, :sentence_len]
                y_temp = X_batch[:, sentence_len]

                class_weights = None
                if SCALE_LOSS_FUN:
                    #class_weights = {}
                    #class_weights[0] = np.where(y_temp == 0)[0].shape[0]/float(len(y_temp))
                    #class_weights[1] = np.where(y_temp == 1)[0].shape[0]/float(len(y_temp))
                    #class_weights[2] = np.where(y_temp == 2)[0].shape[0]/float(len(y_temp))
                    class_weights = class_weight.compute_class_weight('balanced',np.unique(y_temp),y_temp)
                    #print(class_weights)
                try:
                    y_temp = np_utils.to_categorical(y_temp, num_classes=2)
                except Exception as e:
                    print(e)
                    print(y_temp)
                #print(x.shape, y_temp.shape,class_weights)
                loss, acc = model.train_on_batch(x, y_temp, class_weight=class_weights)
                loss_all+=loss
                acc_all+=acc
                count+=1.0
            print('Avg loss:',loss_all/count, 'Avg accuracy:',acc_all/count, 'Epoch:',epoch)
                
        return model
       
    

In [53]:
pd_results = pd.DataFrame(columns=['model+features','kth_fold','precision_train','recall_train','fscore_train','accuracy_train','precision_test','recall_test','fscore_test','accuracy_test'])
count=0

In [ ]:
#####for fine tuning...... fixing the parameters ####### 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedShuffleSplit


TOKENIZER = 'glove_tokenize'
INITIALIZE_WEIGHTS_WITH = 'random'    
LEARN_EMBEDDINGS = False
BATCH_SIZE = 512
HIDDEN=10
LR=0.1
FIXED_EPOCH = 20

np.random.seed(10)
print('embedding: %s' %(GLOVE_MODEL_FILE))
print('Embedding Dimension: %d' %(EMBEDDING_DIM))
print('Allowing embedding learning: %s' %(str(LEARN_EMBEDDINGS)))

#### loading weights and data
SCALE_LOSS_FUN=True
tweets= select_tweets()

#gen_vocab(tweets+sample_list)
gen_vocab(tweets)
#filter_vocab(20000)
X, y = gen_sequence(tweets)
MAX_SEQUENCE_LENGTH = max([len(x) for x in X])
print("max seq length is %d"%(MAX_SEQUENCE_LENGTH))
data = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
y = np.array(y)
#data, y = sklearn.utils.shuffle(data, y)
W = get_embedding_weights()
X=data
sentence_len = X.shape[1]

with open('vocab_lstm.pickle', 'wb') as handle:
     pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
param_sets = list(ParameterGrid(grid_param))
kf = StratifiedKFold(n_splits=10)
fold_count = 0
for train_index, test_index in kf.split(X,y):
    print('cv_fold',fold_count)
    fold_count += 1
    X_train_temp, X_test = X[train_index], X[test_index]
    y_train_temp, y_test = y[train_index], y[test_index]

    y_train_temp = y_train_temp.reshape((len(y_train_temp), 1))

    X_temp = np.hstack((X_train_temp, y_train_temp))

    model = lstm_model(data.shape[1], EMBEDDING_DIM,0.5)
    model=train_LSTM(X_temp ,sentence_len, model, EMBEDDING_DIM, W,epochs=FIXED_EPOCH, batch_size=512,INITIALIZE_WEIGHTS_WITH ="glove")

    y_preds = model.predict(X_test)
    y_pred_train = model.predict(X_train_temp)
    y_preds=[np.argmax(y_pred_1) for y_pred_1 in y_preds]
    y_pred_train=[np.argmax(y_pred_1) for y_pred_1 in y_pred_train]
    print('accuracy_train:',accuracy_score(y_train_temp, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
    df_result_tr,avg_tr=pandas_classification_report(y_train_temp,y_pred_train)
    df_result_te,avg_te=pandas_classification_report(y_test,y_preds) 
    pd_results.loc[count]  = ['GRU_finetuning_glove',fold_count,avg_tr[0],avg_tr[1],avg_tr[2],avg_tr[4],avg_te[0],avg_te[1],avg_te[2],avg_te[4]]
    K.clear_session()
    count=count+1


In [55]:
pd_results.to_csv("lstm_results.csv")